In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 DreamerV3: Imagination-Based Reinforcement Learning with the RSSM

**Notebook 3 of 6 — World Action Models Series | Vizuara AI**

*Estimated time: ~45 minutes*

In this notebook, we will build a Recurrent State-Space Model (RSSM) from scratch, train it as a world model on CartPole, and then train an actor-critic agent entirely inside the world model's imagination — no real environment needed during policy learning.

In [ ]:
# --- Setup and Dependencies ---
!pip install gymnasium -q

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from collections import deque
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Why Does This Matter?

In 2023, a single reinforcement learning algorithm mastered over **150 diverse tasks** — from Atari games to continuous control to 3D navigation — with one set of hyperparameters.

That algorithm was **DreamerV3**.

Even more remarkably, DreamerV3 was the first algorithm to **collect diamonds in Minecraft from scratch** — a task requiring a long chain of subgoals (chop trees, craft planks, build a crafting table, make a wooden pickaxe, mine stone, and so on) with only a sparse reward at the very end.

The secret? DreamerV3 does not learn by endlessly interacting with the real environment. Instead, it builds an internal model of the world and then **trains its policy entirely inside its own imagination.**

By the end of this notebook, you will build an RSSM world model that can imagine future trajectories, and then train an actor-critic agent inside those imaginations — all from scratch in PyTorch.

## 2. Building Intuition

### From Simple RNNs to the RSSM

In the previous notebook, we saw the original World Model by Ha and Schmidhuber (2018). That model used a simple RNN to predict future latent states. The idea was brilliant — but there was a fundamental limitation.

The real world has **both predictable parts and random parts.**

Let us think about this with an analogy.

**Weather forecasting.** Some aspects of weather are highly predictable: the sun will rise tomorrow, summer will be warmer than winter, and high pressure systems move in predictable patterns. A deterministic model captures these regularities beautifully.

But other aspects are genuinely random: will it rain at 3:47 PM on Tuesday? Will a sudden gust of wind knock over your umbrella? These stochastic elements cannot be captured by a purely deterministic model — no matter how large we make it.

The RSSM captures both.

It maintains two parallel streams of information:
1. A **deterministic path** (like a GRU) that captures the predictable, consistent dynamics
2. A **stochastic path** that captures the randomness and uncertainty

Together, they form a more complete picture of the world.

### 🤔 Think About This

Before we proceed, consider this question:

> **Why would a purely deterministic world model fail in a stochastic environment?**

Think about a game where an enemy can appear from the left OR the right with equal probability. A deterministic model must commit to one prediction — say, always predicting the enemy appears from the left. It has no way to express "I am uncertain — the enemy could be on either side."

A stochastic model, on the other hand, can output a probability distribution: 50% left, 50% right. This uncertainty is not a weakness — it is an accurate representation of the world.

## 3. The Mathematics

Now let us formalize these ideas. Brace yourselves — there are three key equations, but we will walk through each one carefully.

### 3.1 The Deterministic Path

$$h_t = f_\theta(h_{t-1}, z_{t-1}, a_{t-1})$$

This equation says: take the previous deterministic hidden state $h_{t-1}$, the previous stochastic latent state $z_{t-1}$, and the previous action $a_{t-1}$, pass them through a recurrent function $f_\theta$ (implemented as a GRU), and produce the new deterministic state $h_t$.

Computationally, this means: concatenate $z_{t-1}$ and $a_{t-1}$ into a single vector, and feed it as input to a GRU cell whose hidden state is $h_{t-1}$. The output is $h_t$ — a compressed summary of everything predictable about the trajectory so far.

**Numerical example.** Suppose $h_{t-1}$ is a vector of dimension 3: $[0.5, -0.2, 0.8]$, $z_{t-1} = [0.1, 0.3]$, and $a_{t-1} = [1.0]$ (a single action). We concatenate $z_{t-1}$ and $a_{t-1}$ to get $[0.1, 0.3, 1.0]$ and feed this as input to the GRU with hidden state $[0.5, -0.2, 0.8]$. The GRU applies its learned gates and produces a new hidden state, say $h_t = [0.7, 0.1, -0.3]$. This new state captures the predictable evolution of the world.

### 3.2 The Encoder (Posterior)

$$z_t \sim q_\theta(z_t \mid h_t, x_t)$$

This equation says: when we have access to a real observation $x_t$, combine it with the deterministic state $h_t$ and compute a probability distribution over the stochastic latent state $z_t$. Then sample $z_t$ from that distribution.

Computationally, this means: concatenate $h_t$ and $x_t$, pass them through an MLP that outputs a mean $\mu$ and standard deviation $\sigma$, and then sample $z_t = \mu + \sigma \cdot \epsilon$ where $\epsilon \sim \mathcal{N}(0, I)$ (the reparameterization trick from VAEs).

This is the **posterior** — it uses real data to compute what the stochastic state should be. We use this during training, when we have access to real observations.

**Numerical example.** Suppose $h_t = [0.7, 0.1, -0.3]$ and $x_t = [1.0, 0.0, -0.5, 0.2]$ (a CartPole observation). We concatenate them to get $[0.7, 0.1, -0.3, 1.0, 0.0, -0.5, 0.2]$ and pass this through an MLP. The MLP outputs $\mu = [0.4, -0.1]$ and $\log\sigma = [-0.5, -0.3]$, giving $\sigma = [0.61, 0.74]$. We sample $\epsilon = [0.2, -0.5]$ from a standard normal, so $z_t = [0.4 + 0.61 \times 0.2,\; -0.1 + 0.74 \times (-0.5)] = [0.52, -0.47]$.

### 3.3 The Prior (Imagination)

$$\hat{z}_t \sim p_\theta(\hat{z}_t \mid h_t)$$

This equation says: when we do **not** have access to a real observation (we are imagining), predict the stochastic state from the deterministic state alone.

Computationally, this means: pass $h_t$ through a different MLP that outputs a mean and standard deviation, and sample $\hat{z}_t$ from that distribution. This is the model's best guess at what the stochastic state should be, without seeing what actually happened.

This is what makes "dreaming" possible. During imagination, we only use the deterministic path and the prior — no real observations are needed.

### 3.4 The KL Divergence Loss

How do we train the prior to be a good predictor? We minimize the KL divergence between the posterior and the prior:

$$\mathcal{L}_{\text{KL}} = D_{\text{KL}}\big[q_\theta(z_t \mid h_t, x_t) \;\|\; p_\theta(\hat{z}_t \mid h_t)\big]$$

This loss pushes the prior (which only sees $h_t$) to match the posterior (which sees both $h_t$ and the real observation $x_t$). As training progresses, the prior gets better at predicting $z_t$ without seeing $x_t$ — which means the model's imagination gets more accurate.

For two Gaussian distributions, the KL divergence has a closed-form solution:

$$D_{\text{KL}} = \log \frac{\sigma_{\text{prior}}}{\sigma_{\text{post}}} + \frac{\sigma_{\text{post}}^2 + (\mu_{\text{post}} - \mu_{\text{prior}})^2}{2\sigma_{\text{prior}}^2} - \frac{1}{2}$$

**Numerical example.** Suppose the posterior has $\mu_{\text{post}} = 0.5$, $\sigma_{\text{post}} = 0.6$ and the prior has $\mu_{\text{prior}} = 0.3$, $\sigma_{\text{prior}} = 0.8$. Then:

$$D_{\text{KL}} = \log\frac{0.8}{0.6} + \frac{0.36 + 0.04}{1.28} - 0.5 = 0.288 + 0.3125 - 0.5 = 0.10$$

A small KL means the prior is already close to the posterior — the model's imagination is accurate. This is exactly what we want.

### 3.5 Full Loss Function

The total world model loss combines three objectives:

$$\mathcal{L} = \mathcal{L}_{\text{obs}} + \mathcal{L}_{\text{reward}} + \beta \cdot \mathcal{L}_{\text{KL}}$$

where $\mathcal{L}_{\text{obs}}$ is the observation reconstruction loss (can the model predict what it sees?), $\mathcal{L}_{\text{reward}}$ is the reward prediction loss, and $\beta$ scales the KL term. We also add a continuation predictor that estimates whether the episode continues.

## 4. Let Us Build It

Now let us implement the RSSM step by step.

### 4.1 Collect CartPole Data

First, we need training data. We will collect trajectories from CartPole using a random policy. This is the same pattern from Notebook 2, but here we are collecting data specifically to train the RSSM.

In [ ]:
# --- Hyperparameters ---
OBS_DIM = 4       # CartPole observation: [cart_pos, cart_vel, pole_angle, pole_vel]
ACTION_DIM = 2    # CartPole actions: left (0) or right (1)
H_DIM = 64        # Deterministic hidden state dimension
Z_DIM = 16        # Stochastic latent state dimension
BATCH_SIZE = 32
SEQ_LEN = 20      # Sequence length for training
LEARNING_RATE = 3e-4
BETA_KL = 0.1     # KL loss weight
NUM_EPISODES = 200
GAMMA = 0.99      # Discount factor for actor-critic
IMAGINE_HORIZON = 15  # Steps to imagine ahead

In [ ]:
def collect_cartpole_data(num_episodes=200, seed=42):
    """Collect trajectories from CartPole with a random policy."""
    env = gym.make('CartPole-v1')
    all_episodes = []

    for ep in range(num_episodes):
        obs, _ = env.reset(seed=seed + ep)
        episode = {'obs': [], 'actions': [], 'rewards': [], 'dones': []}
        done = False

        while not done:
            action = env.action_space.sample()
            episode['obs'].append(obs)
            episode['actions'].append(action)
            next_obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            episode['rewards'].append(reward)
            episode['dones'].append(float(done))
            obs = next_obs

        all_episodes.append(episode)

    env.close()
    return all_episodes

episodes = collect_cartpole_data(NUM_EPISODES)
print(f"Collected {len(episodes)} episodes")
print(f"Average episode length: {np.mean([len(e['rewards']) for e in episodes]):.1f}")

Now let us prepare the data into fixed-length sequences for training.

In [ ]:
def prepare_sequences(episodes, seq_len=20):
    """Convert episodes into fixed-length training sequences."""
    obs_seqs, act_seqs, rew_seqs, done_seqs = [], [], [], []

    for ep in episodes:
        T = len(ep['rewards'])
        if T < seq_len:
            continue
        # Sample a random starting point
        start = np.random.randint(0, T - seq_len + 1)
        obs_seqs.append(ep['obs'][start:start + seq_len])
        act_seqs.append(ep['actions'][start:start + seq_len])
        rew_seqs.append(ep['rewards'][start:start + seq_len])
        done_seqs.append(ep['dones'][start:start + seq_len])

    obs_t = torch.tensor(np.array(obs_seqs), dtype=torch.float32)
    act_t = F.one_hot(torch.tensor(np.array(act_seqs)), ACTION_DIM).float()
    rew_t = torch.tensor(np.array(rew_seqs), dtype=torch.float32)
    done_t = torch.tensor(np.array(done_seqs), dtype=torch.float32)

    return obs_t, act_t, rew_t, done_t

obs_data, act_data, rew_data, done_data = prepare_sequences(episodes, SEQ_LEN)
print(f"Training sequences: {obs_data.shape[0]}")
print(f"Sequence shape: obs={obs_data.shape}, act={act_data.shape}")

### 4.2 Build the RSSM

This is the core of the notebook. The RSSM has three components:

1. **Sequence model** (GRUCell): the deterministic path $h_t = f(h_{t-1}, z_{t-1}, a_{t-1})$
2. **Encoder**: the posterior $q(z_t \mid h_t, x_t)$ — used when we have real observations
3. **Prior**: $p(\hat{z}_t \mid h_t)$ — used during imagination

We use continuous Gaussian latents (rather than the discrete categoricals in the full DreamerV3). This is a design decision: Gaussian latents are simpler to implement and sufficient for CartPole. The full DreamerV3 uses 32 categorical variables with 32 classes each — we will note this difference but keep things approachable.

In [ ]:
class RSSM(nn.Module):
    """
    Recurrent State-Space Model.

    Combines a deterministic recurrent path (GRU) with
    stochastic latent variables (Gaussian) to model both
    predictable and uncertain dynamics.
    """
    def __init__(self, obs_dim, action_dim, h_dim, z_dim):
        super().__init__()
        self.h_dim = h_dim
        self.z_dim = z_dim

        # Deterministic path: GRU takes (z_{t-1}, a_{t-1}) as input
        self.sequence_model = nn.GRUCell(z_dim + action_dim, h_dim)

        # Encoder (posterior): q(z_t | h_t, x_t)
        self.encoder = nn.Sequential(
            nn.Linear(h_dim + obs_dim, 128),
            nn.ELU(),
            nn.Linear(128, z_dim * 2),  # outputs mean and log_std
        )

        # Prior (imagination): p(z_hat_t | h_t)
        self.prior = nn.Sequential(
            nn.Linear(h_dim, 128),
            nn.ELU(),
            nn.Linear(128, z_dim * 2),  # outputs mean and log_std
        )

    def initial_state(self, batch_size):
        """Return zero-initialized hidden state and latent."""
        h = torch.zeros(batch_size, self.h_dim, device=device)
        z = torch.zeros(batch_size, self.z_dim, device=device)
        return h, z

Let us understand this architecture before proceeding.

The `sequence_model` is a GRUCell. At each timestep, it takes the concatenation of $z_{t-1}$ and $a_{t-1}$ as input and updates its hidden state from $h_{t-1}$ to $h_t$. This is the deterministic backbone.

The `encoder` takes the concatenation of $h_t$ and the real observation $x_t$, and outputs the parameters of a Gaussian distribution (mean and log standard deviation). We split the output in half: the first `z_dim` values are the mean, the remaining `z_dim` values are the log standard deviation.

The `prior` does the same thing, but only using $h_t$ — no real observation. This is what the model uses when it is imagining.

Now let us add the methods that sample from these distributions.

In [ ]:
def _sample_gaussian(self, stats):
    """Reparameterized sampling from Gaussian parameters."""
    mean, log_std = torch.chunk(stats, 2, dim=-1)
    std = F.softplus(log_std) + 1e-4  # Ensure positive std
    eps = torch.randn_like(mean)
    z = mean + std * eps  # Reparameterization trick
    return z, mean, std

RSSM._sample_gaussian = _sample_gaussian

### 📊 Visualization: RSSM Architecture

Let us visualize the architecture to solidify our understanding.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
ax.set_xlim(0, 14)
ax.set_ylim(0, 6)
ax.set_aspect('equal')
ax.axis('off')

# --- Draw three timesteps ---
for i, t_label in enumerate(['t-1', 't', 't+1']):
    x_center = 2.5 + i * 4.0

    # Deterministic state (blue box)
    rect_h = plt.Rectangle((x_center - 0.8, 3.8), 1.6, 1.0,
                            facecolor='#3b82f6', edgecolor='black', lw=1.5, alpha=0.85)
    ax.add_patch(rect_h)
    ax.text(x_center, 4.3, f'$h_{{{t_label}}}$', ha='center', va='center',
            fontsize=14, fontweight='bold', color='white')

    # Stochastic state (orange box)
    rect_z = plt.Rectangle((x_center - 0.8, 1.8), 1.6, 1.0,
                            facecolor='#f97316', edgecolor='black', lw=1.5, alpha=0.85)
    ax.add_patch(rect_z)
    ax.text(x_center, 2.3, f'$z_{{{t_label}}}$', ha='center', va='center',
            fontsize=14, fontweight='bold', color='white')

    # Arrow: h -> z (encoder or prior)
    ax.annotate('', xy=(x_center, 2.8), xytext=(x_center, 3.8),
                arrowprops=dict(arrowstyle='->', lw=1.5, color='#6b7280'))

# Arrows between timesteps (deterministic path)
for i in range(2):
    x_start = 2.5 + i * 4.0 + 0.8
    x_end = 2.5 + (i + 1) * 4.0 - 0.8
    ax.annotate('', xy=(x_end, 4.3), xytext=(x_start, 4.3),
                arrowprops=dict(arrowstyle='->', lw=2.5, color='#3b82f6'))
    # Label action + z feeding into next h
    ax.text((x_start + x_end) / 2, 4.75, f'$a, z$', ha='center',
            fontsize=10, color='#374151')

# Observation arrows going into z (encoder path)
for i in range(3):
    x_center = 2.5 + i * 4.0
    ax.annotate('', xy=(x_center - 0.3, 1.8), xytext=(x_center - 0.3, 0.8),
                arrowprops=dict(arrowstyle='->', lw=1.5, color='#10b981',
                                linestyle='dashed'))
    ax.text(x_center - 0.3, 0.5, f'$x_{{{["t-1","t","t+1"][i]}}}$',
            ha='center', fontsize=11, color='#10b981', fontweight='bold')

# Legend
ax.text(0.3, 5.6, 'Deterministic path', fontsize=11, color='#3b82f6', fontweight='bold')
ax.text(0.3, 5.2, 'Stochastic path', fontsize=11, color='#f97316', fontweight='bold')
ax.text(0.3, 4.8, 'Observation (encoder)', fontsize=11, color='#10b981', fontweight='bold')

ax.set_title('RSSM Architecture: Deterministic + Stochastic Paths', fontsize=14, pad=15)
plt.tight_layout()
plt.show()

### 4.3 Implement `observe()` — Processing Real Data

The `observe()` method processes a full sequence of real observations through the RSSM. At each timestep, it:

1. Advances the deterministic state using the GRU
2. Computes the posterior (encoder) distribution using the real observation
3. Computes the prior distribution (for the KL loss)
4. Samples the stochastic state from the posterior

This is how the RSSM processes real experience during training.

In [ ]:
def observe(self, obs_seq, act_seq):
    """
    Process a sequence of real observations through the RSSM.

    Args:
        obs_seq: (batch, seq_len, obs_dim) — real observations
        act_seq: (batch, seq_len, action_dim) — one-hot actions

    Returns:
        Dictionary with all states and distribution parameters
    """
    batch_size, seq_len, _ = obs_seq.shape
    h, z = self.initial_state(batch_size)

    # Storage for outputs
    h_list, z_list = [], []
    post_mean_list, post_std_list = [], []
    prior_mean_list, prior_std_list = [], []

    for t in range(seq_len):
        # Step 1: Deterministic path — advance the GRU
        gz_input = torch.cat([z, act_seq[:, t]], dim=-1)
        h = self.sequence_model(gz_input, h)

        # Step 2: Posterior — use real observation
        post_stats = self.encoder(torch.cat([h, obs_seq[:, t]], dim=-1))
        z, post_mean, post_std = self._sample_gaussian(post_stats)

        # Step 3: Prior — predict without observation
        prior_stats = self.prior(h)
        _, prior_mean, prior_std = self._sample_gaussian(prior_stats)

        # Store everything
        h_list.append(h)
        z_list.append(z)
        post_mean_list.append(post_mean)
        post_std_list.append(post_std)
        prior_mean_list.append(prior_mean)
        prior_std_list.append(prior_std)

    return {
        'h': torch.stack(h_list, dim=1),          # (B, T, h_dim)
        'z': torch.stack(z_list, dim=1),          # (B, T, z_dim)
        'post_mean': torch.stack(post_mean_list, dim=1),
        'post_std': torch.stack(post_std_list, dim=1),
        'prior_mean': torch.stack(prior_mean_list, dim=1),
        'prior_std': torch.stack(prior_std_list, dim=1),
    }

RSSM.observe = observe

### 4.4 Implement `imagine()` — Rolling Forward Without Observations

This is where the magic happens. The `imagine()` method rolls the RSSM forward using only the deterministic path and the prior — **no real observations.** The model is dreaming.

In [ ]:
def imagine(self, initial_h, initial_z, policy_fn, horizon):
    """
    Imagine future trajectories using only the prior (no real observations).

    Args:
        initial_h: (batch, h_dim) — starting deterministic state
        initial_z: (batch, z_dim) — starting stochastic state
        policy_fn: function(h, z) -> action (one-hot)
        horizon: number of steps to imagine ahead

    Returns:
        Dictionary with imagined states
    """
    h, z = initial_h, initial_z
    h_list, z_list = [h], [z]

    for t in range(horizon):
        # Get action from policy
        action = policy_fn(h, z)

        # Step 1: Deterministic path
        gz_input = torch.cat([z, action], dim=-1)
        h = self.sequence_model(gz_input, h)

        # Step 2: Prior only — no real observation available
        prior_stats = self.prior(h)
        z, _, _ = self._sample_gaussian(prior_stats)

        h_list.append(h)
        z_list.append(z)

    return {
        'h': torch.stack(h_list, dim=1),  # (B, H+1, h_dim)
        'z': torch.stack(z_list, dim=1),  # (B, H+1, z_dim)
    }

RSSM.imagine = imagine

Notice the critical difference between `observe()` and `imagine()`:

| | `observe()` (Training) | `imagine()` (Dreaming) |
|---|---|---|
| Deterministic path | Yes (GRU) | Yes (GRU) |
| Stochastic source | Posterior $q(z \mid h, x)$ | Prior $p(\hat{z} \mid h)$ |
| Real observations | Required | **Not needed** |

This is exactly the design that allows DreamerV3 to be so sample-efficient. Once the world model is trained, we can generate unlimited imagined experience at nearly zero cost.

### 📊 Visualization: Observed vs Imagined Trajectories

Let us create a quick test to see the RSSM in action. We will run `observe()` on a real sequence and `imagine()` from the same starting point, then compare.

In [ ]:
# Instantiate the RSSM
rssm = RSSM(OBS_DIM, ACTION_DIM, H_DIM, Z_DIM).to(device)

# Take a small batch for testing
test_obs = obs_data[:4].to(device)
test_act = act_data[:4].to(device)

# Run observe
with torch.no_grad():
    result = rssm.observe(test_obs, test_act)
    print(f"Observed h shape: {result['h'].shape}")
    print(f"Observed z shape: {result['z'].shape}")

# Run imagine from timestep 5
def random_policy(h, z):
    batch = h.shape[0]
    actions = torch.randint(0, ACTION_DIM, (batch,))
    return F.one_hot(actions, ACTION_DIM).float().to(device)

with torch.no_grad():
    init_h = result['h'][:, 5, :]
    init_z = result['z'][:, 5, :]
    imagined = rssm.imagine(init_h, init_z, random_policy, horizon=14)
    print(f"Imagined h shape: {imagined['h'].shape}")

In [ ]:
# Visualize deterministic states: observed vs imagined
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Pick first sequence in batch, first 8 dimensions of h
obs_h = result['h'][0, :, :8].cpu().numpy()
img_h = imagined['h'][0, :, :8].cpu().numpy()

axes[0].imshow(obs_h.T, aspect='auto', cmap='RdBu_r', interpolation='nearest')
axes[0].set_title('Observed Deterministic States $h_t$', fontsize=12)
axes[0].set_xlabel('Timestep')
axes[0].set_ylabel('Hidden Dim (first 8)')

axes[1].imshow(img_h.T, aspect='auto', cmap='RdBu_r', interpolation='nearest')
axes[1].set_title('Imagined Deterministic States $h_t$', fontsize=12)
axes[1].set_xlabel('Timestep')
axes[1].set_ylabel('Hidden Dim (first 8)')

plt.suptitle('RSSM States (before training — random)', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()
print("Note: The model is untrained, so these patterns are random.")
print("After training, imagined states should closely match observed states.")

### 4.5 Reward and Continuation Predictors

DreamerV3 does not just predict states — it also predicts **rewards** and **whether the episode continues.** These predictions are essential for training the actor-critic inside imagination.

In [ ]:
class RewardPredictor(nn.Module):
    """Predicts reward from the RSSM state (h, z)."""
    def __init__(self, h_dim, z_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(h_dim + z_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, 1),
        )

    def forward(self, h, z):
        return self.net(torch.cat([h, z], dim=-1)).squeeze(-1)


class ContinuationPredictor(nn.Module):
    """Predicts whether the episode continues (1 - done probability)."""
    def __init__(self, h_dim, z_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(h_dim + z_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, 1),
        )

    def forward(self, h, z):
        return torch.sigmoid(self.net(torch.cat([h, z], dim=-1))).squeeze(-1)

### 4.6 Actor and Critic for Imagination-Based Training

Now for the most exciting part. The actor and critic are trained **entirely inside the world model's imagination.** They never interact with the real environment.

The **actor** selects actions given the current RSSM state $(h, z)$.

The **critic** estimates the expected return from a given RSSM state.

In [ ]:
class Actor(nn.Module):
    """Policy network: maps RSSM state to action probabilities."""
    def __init__(self, h_dim, z_dim, action_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(h_dim + z_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, action_dim),
        )

    def forward(self, h, z):
        logits = self.net(torch.cat([h, z], dim=-1))
        return logits  # raw logits

    def get_action(self, h, z):
        """Sample action and return one-hot + log_prob."""
        logits = self.forward(h, z)
        dist = torch.distributions.Categorical(logits=logits)
        action_idx = dist.sample()
        log_prob = dist.log_prob(action_idx)
        action_onehot = F.one_hot(action_idx, logits.shape[-1]).float()
        return action_onehot, log_prob


class Critic(nn.Module):
    """Value network: estimates expected return from RSSM state."""
    def __init__(self, h_dim, z_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(h_dim + z_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, 1),
        )

    def forward(self, h, z):
        return self.net(torch.cat([h, z], dim=-1)).squeeze(-1)

Let us take a moment to appreciate the full picture. We have built:

| Component | Purpose | Input | Output |
|---|---|---|---|
| RSSM (GRU) | Deterministic dynamics | $h_{t-1}, z_{t-1}, a_{t-1}$ | $h_t$ |
| Encoder | Posterior (training) | $h_t, x_t$ | $z_t$ |
| Prior | Imagination | $h_t$ | $\hat{z}_t$ |
| Reward Predictor | Reward estimation | $h_t, z_t$ | $\hat{r}_t$ |
| Continuation | Episode termination | $h_t, z_t$ | $\hat{c}_t$ |
| Actor | Action selection | $h_t, z_t$ | $a_t$ |
| Critic | Value estimation | $h_t, z_t$ | $V(h_t, z_t)$ |

## 5. 🔧 Your Turn

Now it is your turn. We have built all the pieces — can you put them together?

### TODO 1: Implement `imagine_ahead()`

This function takes an initial RSSM state, rolls the world model forward for $H$ steps using the actor for actions and the prior for stochastic states, and collects the imagined trajectory with predicted rewards.

In [ ]:
def imagine_ahead(rssm, actor, reward_pred, cont_pred,
                  initial_h, initial_z, horizon):
    """
    Imagine a trajectory of length `horizon` from the given initial state.

    Uses the actor to select actions and the RSSM prior to predict
    future stochastic states. Collects predicted rewards and continuation
    probabilities along the way.

    Args:
        rssm: trained RSSM model
        actor: trained Actor model
        reward_pred: RewardPredictor model
        cont_pred: ContinuationPredictor model
        initial_h: (batch, h_dim) starting deterministic state
        initial_z: (batch, z_dim) starting stochastic state
        horizon: number of steps to imagine

    Returns:
        dict with keys:
            'h': (batch, horizon+1, h_dim) — deterministic states
            'z': (batch, horizon+1, z_dim) — stochastic states
            'rewards': (batch, horizon) — predicted rewards
            'continues': (batch, horizon) — predicted continuation probs
            'log_probs': (batch, horizon) — action log probabilities

    Hints:
        Step 1: Initialize h, z from the inputs. Create empty lists.
        Step 2: Loop for `horizon` steps:
            a) Get action from actor.get_action(h, z)
            b) Advance deterministic state: concat z and action, feed to GRU
            c) Sample stochastic state from the prior
            d) Predict reward and continuation from (h, z)
            e) Append everything to lists
        Step 3: Stack lists into tensors and return
    """
    # ============ TODO ============
    # Implement the imagination loop described above.
    # Use rssm.sequence_model, rssm.prior, rssm._sample_gaussian,
    # actor.get_action, reward_pred, and cont_pred.
    # ==============================

    h, z = initial_h, initial_z
    h_list, z_list = [h], [z]
    reward_list, cont_list, logp_list = [], [], []

    for t in range(horizon):
        action_onehot, log_prob = ???  # YOUR CODE HERE
        gz_input = ???                 # YOUR CODE HERE
        h = ???                        # YOUR CODE HERE
        prior_stats = ???              # YOUR CODE HERE
        z, _, _ = ???                  # YOUR CODE HERE

        rew = ???                      # YOUR CODE HERE
        cont = ???                     # YOUR CODE HERE

        h_list.append(h)
        z_list.append(z)
        reward_list.append(rew)
        cont_list.append(cont)
        logp_list.append(log_prob)

    return {
        'h': torch.stack(h_list, dim=1),
        'z': torch.stack(z_list, dim=1),
        'rewards': torch.stack(reward_list, dim=1),
        'continues': torch.stack(cont_list, dim=1),
        'log_probs': torch.stack(logp_list, dim=1),
    }

In [ ]:
# ✅ Verification: Run this cell to check your imagine_ahead implementation

# Create temporary models for testing
_rssm = RSSM(OBS_DIM, ACTION_DIM, H_DIM, Z_DIM).to(device)
_actor = Actor(H_DIM, Z_DIM, ACTION_DIM).to(device)
_rew_pred = RewardPredictor(H_DIM, Z_DIM).to(device)
_cont_pred = ContinuationPredictor(H_DIM, Z_DIM).to(device)

_h0 = torch.randn(8, H_DIM, device=device)
_z0 = torch.randn(8, Z_DIM, device=device)

try:
    _result = imagine_ahead(_rssm, _actor, _rew_pred, _cont_pred, _h0, _z0, 10)
    assert _result['h'].shape == (8, 11, H_DIM), f"h shape wrong: {_result['h'].shape}"
    assert _result['z'].shape == (8, 11, Z_DIM), f"z shape wrong: {_result['z'].shape}"
    assert _result['rewards'].shape == (8, 10), f"rewards shape wrong: {_result['rewards'].shape}"
    assert _result['continues'].shape == (8, 10), f"continues shape wrong: {_result['continues'].shape}"
    assert _result['log_probs'].shape == (8, 10), f"log_probs shape wrong: {_result['log_probs'].shape}"
    print("All shape checks passed!")
    print("rewards sample:", _result['rewards'][0, :3].detach().cpu().numpy())
    print("continues sample:", _result['continues'][0, :3].detach().cpu().numpy())
except Exception as e:
    print(f"Error: {e}")
    print("Check your implementation and try again.")

del _rssm, _actor, _rew_pred, _cont_pred

### TODO 2: Implement the Actor Loss

The actor is trained using imagined value estimates. The idea is simple: imagine trajectories, compute returns using the critic, and update the actor to select actions that lead to higher imagined returns.

We use a simplified **REINFORCE-style policy gradient** on imagined trajectories:

$$\mathcal{L}_{\text{actor}} = -\frac{1}{H} \sum_{t=0}^{H-1} \log \pi(a_t \mid h_t, z_t) \cdot \text{sg}(R_t - V(h_t, z_t))$$

where $R_t$ is the lambda-return computed from imagined rewards and $\text{sg}$ means stop-gradient (we do not backpropagate through the returns).

In [ ]:
def compute_actor_loss(log_probs, rewards, continues, values, gamma=0.99):
    """
    Compute the actor (policy gradient) loss from imagined trajectories.

    Args:
        log_probs: (batch, horizon) — log probabilities of chosen actions
        rewards: (batch, horizon) — predicted rewards
        continues: (batch, horizon) — predicted continuation probabilities
        values: (batch, horizon+1) — critic values for all states

    Returns:
        Scalar loss (to be minimized)

    Hints:
        Step 1: Compute discounted returns working BACKWARDS from the last step.
                R_t = r_t + gamma * c_t * R_{t+1}
                Initialize R_H = values[:, -1] (bootstrap from critic)
        Step 2: Compute advantages: A_t = R_t - values[:, t] (detached)
        Step 3: Policy gradient loss: -mean(log_prob * advantage)
    """
    batch_size, horizon = rewards.shape

    # ============ TODO ============
    # Step 1: Compute returns backwards
    returns = torch.zeros(batch_size, horizon, device=rewards.device)
    R = ???  # Bootstrap: use the critic's value at the last imagined state

    for t in reversed(range(horizon)):
        R = ???  # Bellman backup with continuation probability
        returns[:, t] = R

    # Step 2: Compute advantages (stop gradient on returns and values)
    advantages = ???  # YOUR CODE HERE

    # Step 3: Policy gradient loss
    actor_loss = ???  # YOUR CODE HERE
    # ==============================

    return actor_loss

In [ ]:
# ✅ Verification: Run this cell to check your actor loss

_log_probs = torch.randn(4, 10)
_rewards = torch.ones(4, 10)       # constant reward of 1
_continues = torch.ones(4, 10)     # never terminates
_values = torch.zeros(4, 11)       # critic predicts 0

try:
    _loss = compute_actor_loss(_log_probs, _rewards, _continues, _values, gamma=0.99)
    assert _loss.dim() == 0, f"Loss should be scalar, got shape {_loss.shape}"
    # With constant rewards=1, continues=1, values=0, gamma=0.99:
    # Returns should be large positive numbers → advantages are positive
    # Actor loss should be finite
    assert torch.isfinite(_loss), f"Loss is not finite: {_loss}"
    print(f"Actor loss: {_loss.item():.4f}")
    print("Loss is finite and scalar — implementation looks correct!")
except Exception as e:
    print(f"Error: {e}")
    print("Check your implementation and try again.")

## 6. Putting It All Together

Now let us build the complete training pipeline. We train in two phases:

**Phase 1: Train the world model** on real data (observation reconstruction + KL divergence).

**Phase 2: Train the actor-critic** inside the world model's imagination.

In [ ]:
def kl_divergence_gaussian(post_mean, post_std, prior_mean, prior_std):
    """
    Compute KL divergence between two diagonal Gaussians.
    KL(q || p) for each dimension, then sum over the latent dims.
    """
    var_ratio = (post_std / prior_std).pow(2)
    mean_diff = (post_mean - prior_mean).pow(2) / prior_std.pow(2)
    kl = 0.5 * (var_ratio + mean_diff - 1.0 - var_ratio.log())
    return kl.sum(dim=-1)  # Sum over latent dimensions

In [ ]:
class ObservationDecoder(nn.Module):
    """Reconstructs observations from RSSM state (h, z)."""
    def __init__(self, h_dim, z_dim, obs_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(h_dim + z_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ELU(),
            nn.Linear(hidden_dim, obs_dim),
        )

    def forward(self, h, z):
        return self.net(torch.cat([h, z], dim=-1))

In [ ]:
# --- Instantiate all models ---
rssm = RSSM(OBS_DIM, ACTION_DIM, H_DIM, Z_DIM).to(device)
obs_decoder = ObservationDecoder(H_DIM, Z_DIM, OBS_DIM).to(device)
reward_pred = RewardPredictor(H_DIM, Z_DIM).to(device)
cont_pred = ContinuationPredictor(H_DIM, Z_DIM).to(device)
actor = Actor(H_DIM, Z_DIM, ACTION_DIM).to(device)
critic = Critic(H_DIM, Z_DIM).to(device)

# --- Optimizers ---
world_model_params = (
    list(rssm.parameters()) +
    list(obs_decoder.parameters()) +
    list(reward_pred.parameters()) +
    list(cont_pred.parameters())
)
world_opt = optim.Adam(world_model_params, lr=LEARNING_RATE)
actor_opt = optim.Adam(actor.parameters(), lr=LEARNING_RATE)
critic_opt = optim.Adam(critic.parameters(), lr=LEARNING_RATE)

print(f"World model parameters: {sum(p.numel() for p in world_model_params):,}")
print(f"Actor parameters: {sum(p.numel() for p in actor.parameters()):,}")
print(f"Critic parameters: {sum(p.numel() for p in critic.parameters()):,}")

## 7. Training and Results

### Phase 1: World Model Training

We train the RSSM to reconstruct observations, predict rewards, predict episode continuation, and minimize the KL divergence between the posterior and prior.

In [ ]:
def train_world_model_step(rssm, obs_decoder, reward_pred, cont_pred,
                           obs_batch, act_batch, rew_batch, done_batch,
                           optimizer, beta_kl=0.1):
    """One training step for the world model."""
    optimizer.zero_grad()

    # Forward pass through the RSSM
    result = rssm.observe(obs_batch, act_batch)
    h, z = result['h'], result['z']

    # Observation reconstruction loss
    obs_pred = obs_decoder(h, z)
    obs_loss = F.mse_loss(obs_pred, obs_batch)

    # Reward prediction loss
    rew_pred = reward_pred(h, z)
    rew_loss = F.mse_loss(rew_pred, rew_batch)

    # Continuation prediction loss (binary cross-entropy)
    cont_target = 1.0 - done_batch  # continue = 1 - done
    cont_logit = cont_pred(h, z)
    cont_loss = F.binary_cross_entropy(cont_logit, cont_target)

    # KL divergence loss
    kl_loss = kl_divergence_gaussian(
        result['post_mean'], result['post_std'],
        result['prior_mean'], result['prior_std']
    ).mean()

    # Total loss
    total_loss = obs_loss + rew_loss + cont_loss + beta_kl * kl_loss
    total_loss.backward()
    nn.utils.clip_grad_norm_(world_model_params, max_norm=100.0)
    optimizer.step()

    return {
        'total': total_loss.item(),
        'obs': obs_loss.item(),
        'reward': rew_loss.item(),
        'kl': kl_loss.item(),
        'cont': cont_loss.item(),
    }

In [ ]:
# --- Phase 1: Train the World Model ---
NUM_WORLD_EPOCHS = 80
world_losses = {'total': [], 'obs': [], 'reward': [], 'kl': [], 'cont': []}
n_sequences = obs_data.shape[0]

print("Phase 1: Training World Model...")
for epoch in range(NUM_WORLD_EPOCHS):
    # Re-sample sequences each epoch for diversity
    obs_data, act_data, rew_data, done_data = prepare_sequences(episodes, SEQ_LEN)

    # Shuffle and batch
    perm = torch.randperm(obs_data.shape[0])
    epoch_losses = {k: [] for k in world_losses}

    for i in range(0, len(perm) - BATCH_SIZE + 1, BATCH_SIZE):
        idx = perm[i:i + BATCH_SIZE]
        obs_b = obs_data[idx].to(device)
        act_b = act_data[idx].to(device)
        rew_b = rew_data[idx].to(device)
        done_b = done_data[idx].to(device)

        losses = train_world_model_step(
            rssm, obs_decoder, reward_pred, cont_pred,
            obs_b, act_b, rew_b, done_b, world_opt, BETA_KL
        )
        for k, v in losses.items():
            epoch_losses[k].append(v)

    for k in world_losses:
        world_losses[k].append(np.mean(epoch_losses[k]))

    if (epoch + 1) % 20 == 0:
        print(f"  Epoch {epoch+1:3d}/{NUM_WORLD_EPOCHS} | "
              f"Loss: {world_losses['total'][-1]:.4f} | "
              f"Obs: {world_losses['obs'][-1]:.4f} | "
              f"KL: {world_losses['kl'][-1]:.4f}")

print("World model training complete!")

### 📊 World Model Training Curves

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0, 0].plot(world_losses['total'], color='#2563eb', lw=2)
axes[0, 0].set_title('Total Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(world_losses['obs'], color='#16a34a', lw=2)
axes[0, 1].set_title('Observation Reconstruction Loss', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(world_losses['kl'], color='#ea580c', lw=2)
axes[1, 0].set_title('KL Divergence', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(world_losses['reward'], color='#9333ea', lw=2)
axes[1, 1].set_title('Reward Prediction Loss', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('World Model Training', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### Phase 2: Actor-Critic Training in Imagination

Now the exciting part. We freeze the world model and train the actor-critic **entirely inside the RSSM's imagination.** No real environment interaction.

The training loop works as follows:
1. Sample a batch of real sequences and run them through the RSSM to get starting states
2. From those starting states, imagine forward using the actor and prior
3. Predict rewards and continuation probabilities along the imagined trajectory
4. Train the critic to estimate returns
5. Train the actor to maximize those returns

In [ ]:
def train_actor_critic_step(rssm, actor, critic, reward_pred, cont_pred,
                            start_h, start_z, actor_opt, critic_opt,
                            horizon=15, gamma=0.99):
    """One training step for actor-critic in imagination."""

    # --- Imagine trajectories ---
    h, z = start_h.detach(), start_z.detach()
    h_traj, z_traj = [h], [z]
    log_probs_list = []

    for t in range(horizon):
        action_oh, log_prob = actor.get_action(h, z)
        gz_input = torch.cat([z, action_oh], dim=-1)
        h = rssm.sequence_model(gz_input, h)
        prior_stats = rssm.prior(h)
        z, _, _ = rssm._sample_gaussian(prior_stats)
        h_traj.append(h)
        z_traj.append(z)
        log_probs_list.append(log_prob)

    h_traj = torch.stack(h_traj, dim=1)   # (B, H+1, h_dim)
    z_traj = torch.stack(z_traj, dim=1)   # (B, H+1, z_dim)
    log_probs = torch.stack(log_probs_list, dim=1)  # (B, H)

    # --- Predict rewards and continuation ---
    with torch.no_grad():
        rewards = reward_pred(h_traj[:, 1:], z_traj[:, 1:])   # (B, H)
        continues = cont_pred(h_traj[:, 1:], z_traj[:, 1:])   # (B, H)

    # --- Train Critic ---
    values = critic(h_traj.detach(), z_traj.detach())  # (B, H+1)

    # Compute targets (discounted returns)
    with torch.no_grad():
        targets = torch.zeros_like(rewards)
        R = values[:, -1]
        for t in reversed(range(horizon)):
            R = rewards[:, t] + gamma * continues[:, t] * R
            targets[:, t] = R

    critic_loss = F.mse_loss(values[:, :-1], targets)
    critic_opt.zero_grad()
    critic_loss.backward()
    nn.utils.clip_grad_norm_(critic.parameters(), max_norm=100.0)
    critic_opt.step()

    # --- Train Actor ---
    values_detached = critic(h_traj, z_traj).detach()  # fresh forward pass
    advantages = targets - values_detached[:, :-1]
    actor_loss = -(log_probs * advantages.detach()).mean()

    actor_opt.zero_grad()
    actor_loss.backward()
    nn.utils.clip_grad_norm_(actor.parameters(), max_norm=100.0)
    actor_opt.step()

    return actor_loss.item(), critic_loss.item()

In [ ]:
# --- Phase 2: Train Actor-Critic in Imagination ---
NUM_AC_EPOCHS = 120
actor_losses, critic_losses = [], []

print("Phase 2: Training Actor-Critic in Imagination...")
for epoch in range(NUM_AC_EPOCHS):
    # Get starting states from real data
    obs_data_fresh, act_data_fresh, _, _ = prepare_sequences(episodes, SEQ_LEN)
    perm = torch.randperm(obs_data_fresh.shape[0])
    epoch_actor, epoch_critic = [], []

    for i in range(0, len(perm) - BATCH_SIZE + 1, BATCH_SIZE):
        idx = perm[i:i + BATCH_SIZE]
        obs_b = obs_data_fresh[idx].to(device)
        act_b = act_data_fresh[idx].to(device)

        # Get RSSM states from real data (frozen world model)
        with torch.no_grad():
            result = rssm.observe(obs_b, act_b)

        # Pick a random starting timestep
        t0 = np.random.randint(0, SEQ_LEN - 1)
        start_h = result['h'][:, t0, :]
        start_z = result['z'][:, t0, :]

        a_loss, c_loss = train_actor_critic_step(
            rssm, actor, critic, reward_pred, cont_pred,
            start_h, start_z, actor_opt, critic_opt,
            IMAGINE_HORIZON, GAMMA
        )
        epoch_actor.append(a_loss)
        epoch_critic.append(c_loss)

    actor_losses.append(np.mean(epoch_actor))
    critic_losses.append(np.mean(epoch_critic))

    if (epoch + 1) % 30 == 0:
        print(f"  Epoch {epoch+1:3d}/{NUM_AC_EPOCHS} | "
              f"Actor: {actor_losses[-1]:.4f} | "
              f"Critic: {critic_losses[-1]:.4f}")

print("Actor-critic training complete!")

### 📊 Actor-Critic Training Curves

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(actor_losses, color='#dc2626', lw=2, label='Actor Loss')
axes[0].set_title('Actor Loss (Policy Gradient)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].grid(True, alpha=0.3)

axes[1].plot(critic_losses, color='#2563eb', lw=2, label='Critic Loss')
axes[1].set_title('Critic Loss (Value Estimation)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Actor-Critic Training (Inside Imagination)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 8. 🎯 Final Output: Real vs Imagined Trajectories

Now let us see the trained RSSM in action. We will:

1. Collect a real trajectory from CartPole using the learned actor
2. From the same initial state, imagine a trajectory using only the RSSM
3. Compare the two side by side

This is the ultimate test: does the world model's imagination match reality?

In [ ]:
def run_real_episode(actor, rssm, max_steps=200):
    """Run a real CartPole episode using the learned actor."""
    env = gym.make('CartPole-v1')
    obs, _ = env.reset(seed=123)

    h = torch.zeros(1, H_DIM, device=device)
    z = torch.zeros(1, Z_DIM, device=device)

    real_obs_list = [obs.copy()]
    actions_taken = []
    rewards = []

    for step in range(max_steps):
        obs_t = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)

        # Update RSSM state with real observation
        if step > 0:
            act_t = F.one_hot(
                torch.tensor([actions_taken[-1]], device=device), ACTION_DIM
            ).float()
            gz_input = torch.cat([z, act_t], dim=-1)
            h = rssm.sequence_model(gz_input, h)

        # Encode with real observation
        post_stats = rssm.encoder(torch.cat([h, obs_t], dim=-1))
        z, _, _ = rssm._sample_gaussian(post_stats)

        # Select action
        with torch.no_grad():
            logits = actor(h, z)
            action = torch.argmax(logits, dim=-1).item()

        obs, reward, terminated, truncated, _ = env.step(action)
        real_obs_list.append(obs.copy())
        actions_taken.append(action)
        rewards.append(reward)

        if terminated or truncated:
            break

    env.close()
    return np.array(real_obs_list), actions_taken, rewards

In [ ]:
def run_imagined_episode(rssm, actor, reward_pred, initial_obs,
                         actions, max_steps=200):
    """Imagine a trajectory from an initial observation, replaying the same actions."""
    h = torch.zeros(1, H_DIM, device=device)
    z = torch.zeros(1, Z_DIM, device=device)

    # Encode the initial observation
    obs_t = torch.tensor(initial_obs, dtype=torch.float32, device=device).unsqueeze(0)
    post_stats = rssm.encoder(torch.cat([h, obs_t], dim=-1))
    z, _, _ = rssm._sample_gaussian(post_stats)

    imagined_rewards = []
    imagined_h_states = [h.squeeze(0).detach().cpu().numpy()]

    steps = min(len(actions), max_steps)
    for step in range(steps):
        act_t = F.one_hot(
            torch.tensor([actions[step]], device=device), ACTION_DIM
        ).float()

        # Deterministic step
        gz_input = torch.cat([z, act_t], dim=-1)
        h = rssm.sequence_model(gz_input, h)

        # Prior only — no real observation
        prior_stats = rssm.prior(h)
        z, _, _ = rssm._sample_gaussian(prior_stats)

        # Predict reward
        with torch.no_grad():
            rew = reward_pred(h, z).item()

        imagined_rewards.append(rew)
        imagined_h_states.append(h.squeeze(0).detach().cpu().numpy())

    return np.array(imagined_h_states), imagined_rewards

In [ ]:
# Run real episode
real_obs, real_actions, real_rewards = run_real_episode(actor, rssm)
print(f"Real episode length: {len(real_rewards)} steps")
print(f"Real total reward: {sum(real_rewards):.0f}")

# Run imagined episode from the same starting point
img_h_states, img_rewards = run_imagined_episode(
    rssm, actor, reward_pred, real_obs[0], real_actions
)
print(f"Imagined episode length: {len(img_rewards)} steps")
print(f"Imagined total reward: {sum(img_rewards):.2f}")

In [ ]:
# --- Compare real vs imagined rewards ---
fig, axes = plt.subplots(2, 2, figsize=(14, 9))

# Plot 1: Reward comparison
steps = min(len(real_rewards), len(img_rewards))
axes[0, 0].plot(range(steps), real_rewards[:steps],
                'b-', lw=2, label='Real Reward', alpha=0.8)
axes[0, 0].plot(range(steps), img_rewards[:steps],
                'r--', lw=2, label='Imagined Reward', alpha=0.8)
axes[0, 0].set_title('Real vs Imagined Rewards', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Timestep')
axes[0, 0].set_ylabel('Reward')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Cumulative reward comparison
real_cumsum = np.cumsum(real_rewards[:steps])
img_cumsum = np.cumsum(img_rewards[:steps])
axes[0, 1].plot(range(steps), real_cumsum, 'b-', lw=2, label='Real Cumulative')
axes[0, 1].plot(range(steps), img_cumsum, 'r--', lw=2, label='Imagined Cumulative')
axes[0, 1].set_title('Cumulative Reward', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Timestep')
axes[0, 1].set_ylabel('Cumulative Reward')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Real observation trajectories (CartPole state variables)
obs_labels = ['Cart Position', 'Cart Velocity', 'Pole Angle', 'Pole Velocity']
colors = ['#2563eb', '#16a34a', '#ea580c', '#9333ea']
for j in range(4):
    axes[1, 0].plot(real_obs[:steps, j], color=colors[j],
                     lw=1.5, label=obs_labels[j], alpha=0.8)
axes[1, 0].set_title('Real CartPole Observations', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Timestep')
axes[1, 0].legend(fontsize=8, loc='upper left')
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Imagined hidden states (first 4 dims)
img_h_arr = np.array(img_h_states[:steps])
for j in range(min(4, img_h_arr.shape[1])):
    axes[1, 1].plot(img_h_arr[:, j], color=colors[j],
                     lw=1.5, label=f'h dim {j}', alpha=0.8)
axes[1, 1].set_title('Imagined Deterministic States $h_t$', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Timestep')
axes[1, 1].legend(fontsize=8, loc='upper left')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Real Environment vs RSSM Imagination', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

### How Imagination Improves with Training

Let us also visualize how the quality of the imagined rewards changed as the world model trained. We will measure the reward prediction error at different training checkpoints.

In [ ]:
# Evaluate imagination quality: predicted vs actual reward on held-out data
with torch.no_grad():
    # Take a fresh batch of real sequences
    eval_obs, eval_act, eval_rew, _ = prepare_sequences(episodes, SEQ_LEN)
    eval_obs = eval_obs[:16].to(device)
    eval_act = eval_act[:16].to(device)
    eval_rew = eval_rew[:16].to(device)

    result = rssm.observe(eval_obs, eval_act)
    pred_rew = reward_pred(result['h'], result['z'])

    # Per-timestep comparison
    mean_real = eval_rew.mean(dim=0).cpu().numpy()
    mean_pred = pred_rew.mean(dim=0).cpu().numpy()

fig, ax = plt.subplots(figsize=(10, 4))
t_range = range(SEQ_LEN)
ax.plot(t_range, mean_real, 'bo-', lw=2, markersize=5, label='Real Reward')
ax.plot(t_range, mean_pred, 'r^--', lw=2, markersize=5, label='Predicted Reward')
ax.fill_between(t_range, mean_real, mean_pred, alpha=0.15, color='purple')
ax.set_title('Reward Prediction: Real vs World Model', fontsize=13, fontweight='bold')
ax.set_xlabel('Timestep in Sequence', fontsize=11)
ax.set_ylabel('Reward', fontsize=11)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

mae = np.mean(np.abs(mean_real - mean_pred))
print(f"Mean Absolute Reward Prediction Error: {mae:.4f}")

In [ ]:
# --- Final evaluation: run multiple episodes with the learned actor ---
print("=" * 60)
print("  FINAL EVALUATION: DreamerV3-Style Actor on CartPole")
print("=" * 60)

eval_rewards = []
for ep in range(20):
    obs_list, _, rewards = run_real_episode(actor, rssm)
    eval_rewards.append(sum(rewards))

mean_reward = np.mean(eval_rewards)
std_reward = np.std(eval_rewards)
max_reward = np.max(eval_rewards)

print(f"\n  Episodes evaluated: 20")
print(f"  Mean reward: {mean_reward:.1f} +/- {std_reward:.1f}")
print(f"  Max reward:  {max_reward:.0f}")
print(f"  Min reward:  {np.min(eval_rewards):.0f}")

print("\n" + "=" * 60)
print("  Congratulations!")
print("  You built an RSSM world model from scratch,")
print("  trained it on CartPole data, and then trained")
print("  an actor-critic entirely inside the model's")
print("  imagination. This is the core idea behind DreamerV3.")
print("=" * 60)

In [ ]:
# Bar plot of evaluation episodes
fig, ax = plt.subplots(figsize=(10, 4))
bars = ax.bar(range(20), eval_rewards, color=['#3b82f6' if r >= mean_reward
              else '#f97316' for r in eval_rewards], edgecolor='black', lw=0.5)
ax.axhline(y=mean_reward, color='red', linestyle='--', lw=2,
           label=f'Mean: {mean_reward:.1f}')
ax.set_title('Evaluation: Actor Trained in Imagination', fontsize=13, fontweight='bold')
ax.set_xlabel('Episode', fontsize=11)
ax.set_ylabel('Total Reward', fontsize=11)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 9. Reflection

Take a moment to reflect on what we have built. The key insight is profound: once you have a good world model, you can train a policy **without any real environment interaction.** This is what makes DreamerV3 so sample-efficient.

### 🤔 Reflection Questions

1. **Why does the RSSM use both a deterministic and stochastic path?** Could we achieve the same result with just one of them? What would we lose?

2. **The KL divergence loss pushes the prior toward the posterior.** What would happen if we removed this loss entirely? How would imagination quality be affected?

3. **DreamerV3 uses discrete categorical latents (32 variables x 32 classes) instead of continuous Gaussians.** Why might discrete latents be better for modeling complex environments? (Hint: think about multimodal distributions — the ball could go left OR right.)

### Optional Challenges

**Challenge 1: Discrete Latents.** Replace the Gaussian stochastic path with categorical distributions. Use `torch.distributions.OneHotCategorical` with straight-through gradients. Compare the KL divergence behavior.

**Challenge 2: Symlog Predictions.** DreamerV3 uses a "symlog" transformation for predictions: $\text{symlog}(x) = \text{sign}(x) \ln(|x| + 1)$. Implement this for the reward predictor and see if it stabilizes training on environments with varying reward scales.

### What Comes Next

In this notebook, we built the RSSM — the world model that powers DreamerV3's imagination. But DreamerV3 is just one approach to learning about the world.

In the next notebook, we will explore a fundamentally different philosophy: **what if we do not need to predict pixels at all?** Yann LeCun's Joint Embedding Predictive Architecture (JEPA) argues that predicting in abstract representation space is both more efficient and more aligned with how biological brains work.

The question is: can we build a world model that understands without reconstructing?

We will find out in **Notebook 4: JEPA — Predicting in Abstract Space.**

See you there!

In [ ]:
#@title 💬 AI Teaching Assistant — Click ▶ to start
#@markdown This AI chatbot reads your notebook and can answer questions about any concept, code, or exercise.

import json as _json
import requests as _requests
from google.colab import output as _output
from IPython.display import display, HTML as _HTML, Markdown as _Markdown

# --- Read notebook content for context ---
def _get_notebook_context():
    try:
        from google.colab import _message
        nb = _message.blocking_request("get_ipynb", request="", timeout_sec=10)
        cells = nb.get("ipynb", {}).get("cells", [])
        parts = []
        for cell in cells:
            src = "".join(cell.get("source", []))
            tags = cell.get("metadata", {}).get("tags", [])
            if "chatbot" in tags:
                continue
            if src.strip():
                ct = cell.get("cell_type", "unknown")
                parts.append(f"[{ct.upper()}]\n{src}")
        return "\n\n---\n\n".join(parts)
    except Exception:
        return "Notebook content unavailable."

_NOTEBOOK_CONTEXT = _get_notebook_context()
_CHAT_HISTORY = []
_API_URL = "https://course-creator-brown.vercel.app/api/chat"

def _notebook_chat(question):
    global _CHAT_HISTORY
    try:
        resp = _requests.post(_API_URL, json={
            'question': question,
            'context': _NOTEBOOK_CONTEXT[:100000],
            'history': _CHAT_HISTORY[-10:],
        }, timeout=60)
        data = resp.json()
        answer = data.get('answer', 'Sorry, I could not generate a response.')
        _CHAT_HISTORY.append({'role': 'user', 'content': question})
        _CHAT_HISTORY.append({'role': 'assistant', 'content': answer})
        return answer
    except Exception as e:
        return f'Error connecting to teaching assistant: {str(e)}'

_output.register_callback('notebook_chat', _notebook_chat)

def ask(question):
    """Ask the AI teaching assistant a question about this notebook."""
    answer = _notebook_chat(question)
    display(_Markdown(answer))

print("\u2705 AI Teaching Assistant is ready!")
print("\U0001f4a1 Use the chat below, or call ask(\'your question\') in any cell.")

# --- Display chat widget ---
display(_HTML('''<style>
  .vc-wrap{font-family:-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,sans-serif;max-width:100%;border-radius:16px;overflow:hidden;box-shadow:0 4px 24px rgba(0,0,0,.12);background:#fff;border:1px solid #e5e7eb}
  .vc-hdr{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;padding:16px 20px;display:flex;align-items:center;gap:12px}
  .vc-avatar{width:42px;height:42px;background:rgba(255,255,255,.2);border-radius:50%;display:flex;align-items:center;justify-content:center;font-size:22px}
  .vc-hdr h3{font-size:16px;font-weight:600;margin:0}
  .vc-hdr p{font-size:12px;opacity:.85;margin:2px 0 0}
  .vc-msgs{height:420px;overflow-y:auto;padding:16px;background:#f8f9fb;display:flex;flex-direction:column;gap:10px}
  .vc-msg{display:flex;flex-direction:column;animation:vc-fade .25s ease}
  .vc-msg.user{align-items:flex-end}
  .vc-msg.bot{align-items:flex-start}
  .vc-bbl{max-width:85%;padding:10px 14px;border-radius:16px;font-size:14px;line-height:1.55;word-wrap:break-word}
  .vc-msg.user .vc-bbl{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border-bottom-right-radius:4px}
  .vc-msg.bot .vc-bbl{background:#fff;color:#1a1a2e;border:1px solid #e8e8e8;border-bottom-left-radius:4px}
  .vc-bbl code{background:rgba(0,0,0,.07);padding:2px 6px;border-radius:4px;font-size:13px;font-family:'Fira Code',monospace}
  .vc-bbl pre{background:#1e1e2e;color:#cdd6f4;padding:12px;border-radius:8px;overflow-x:auto;margin:8px 0;font-size:13px}
  .vc-bbl pre code{background:none;padding:0;color:inherit}
  .vc-bbl h3,.vc-bbl h4{margin:10px 0 4px;font-size:15px}
  .vc-bbl ul,.vc-bbl ol{margin:4px 0;padding-left:20px}
  .vc-bbl li{margin:2px 0}
  .vc-chips{display:flex;flex-wrap:wrap;gap:8px;padding:0 16px 12px;background:#f8f9fb}
  .vc-chip{background:#fff;border:1px solid #d1d5db;border-radius:20px;padding:6px 14px;font-size:12px;cursor:pointer;transition:all .15s;color:#4b5563}
  .vc-chip:hover{border-color:#667eea;color:#667eea;background:#f0f0ff}
  .vc-input{display:flex;padding:12px 16px;background:#fff;border-top:1px solid #eee;gap:8px}
  .vc-input input{flex:1;padding:10px 16px;border:2px solid #e8e8e8;border-radius:24px;font-size:14px;outline:none;transition:border-color .2s}
  .vc-input input:focus{border-color:#667eea}
  .vc-input button{background:linear-gradient(135deg,#667eea 0%,#764ba2 100%);color:#fff;border:none;border-radius:50%;width:42px;height:42px;cursor:pointer;display:flex;align-items:center;justify-content:center;font-size:18px;transition:transform .1s}
  .vc-input button:hover{transform:scale(1.05)}
  .vc-input button:disabled{opacity:.5;cursor:not-allowed;transform:none}
  .vc-typing{display:flex;gap:5px;padding:4px 0}
  .vc-typing span{width:8px;height:8px;background:#667eea;border-radius:50%;animation:vc-bounce 1.4s infinite ease-in-out}
  .vc-typing span:nth-child(2){animation-delay:.2s}
  .vc-typing span:nth-child(3){animation-delay:.4s}
  @keyframes vc-bounce{0%,80%,100%{transform:scale(0)}40%{transform:scale(1)}}
  @keyframes vc-fade{from{opacity:0;transform:translateY(8px)}to{opacity:1;transform:translateY(0)}}
  .vc-note{text-align:center;font-size:11px;color:#9ca3af;padding:8px 16px 12px;background:#fff}
</style>
<div class="vc-wrap">
  <div class="vc-hdr">
    <div class="vc-avatar">&#129302;</div>
    <div>
      <h3>Vizuara Teaching Assistant</h3>
      <p>Ask me anything about this notebook</p>
    </div>
  </div>
  <div class="vc-msgs" id="vcMsgs">
    <div class="vc-msg bot">
      <div class="vc-bbl">&#128075; Hi! I've read through this entire notebook. Ask me about any concept, code block, or exercise &mdash; I'm here to help you learn!</div>
    </div>
  </div>
  <div class="vc-chips" id="vcChips">
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Explain the main concept</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Help with the TODO exercise</span>
    <span class="vc-chip" onclick="vcAsk(this.textContent)">Summarize what I learned</span>
  </div>
  <div class="vc-input">
    <input type="text" id="vcIn" placeholder="Ask about concepts, code, exercises..." />
    <button id="vcSend" onclick="vcSendMsg()">&#10148;</button>
  </div>
  <div class="vc-note">AI-generated &middot; Verify important information &middot; <a href="#" onclick="vcClear();return false" style="color:#667eea">Clear chat</a></div>
</div>
<script>
(function(){
  var msgs=document.getElementById('vcMsgs'),inp=document.getElementById('vcIn'),
      btn=document.getElementById('vcSend'),chips=document.getElementById('vcChips');

  function esc(s){var d=document.createElement('div');d.textContent=s;return d.innerHTML}

  function md(t){
    return t
      .replace(/```(\w*)\n([\s\S]*?)```/g,function(_,l,c){return '<pre><code>'+esc(c)+'</code></pre>'})
      .replace(/`([^`]+)`/g,'<code>$1</code>')
      .replace(/\*\*([^*]+)\*\*/g,'<strong>$1</strong>')
      .replace(/\*([^*]+)\*/g,'<em>$1</em>')
      .replace(/^#### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^### (.+)$/gm,'<h4>$1</h4>')
      .replace(/^## (.+)$/gm,'<h3>$1</h3>')
      .replace(/^\d+\. (.+)$/gm,'<li>$1</li>')
      .replace(/^- (.+)$/gm,'<li>$1</li>')
      .replace(/\n\n/g,'<br><br>')
      .replace(/\n/g,'<br>');
  }

  function addMsg(text,isUser){
    var m=document.createElement('div');m.className='vc-msg '+(isUser?'user':'bot');
    var b=document.createElement('div');b.className='vc-bbl';
    b.innerHTML=isUser?esc(text):md(text);
    m.appendChild(b);msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function showTyping(){
    var m=document.createElement('div');m.className='vc-msg bot';m.id='vcTyping';
    m.innerHTML='<div class="vc-bbl"><div class="vc-typing"><span></span><span></span><span></span></div></div>';
    msgs.appendChild(m);msgs.scrollTop=msgs.scrollHeight;
  }

  function hideTyping(){var e=document.getElementById('vcTyping');if(e)e.remove()}

  window.vcSendMsg=function(){
    var q=inp.value.trim();if(!q)return;
    inp.value='';chips.style.display='none';
    addMsg(q,true);showTyping();btn.disabled=true;
    google.colab.kernel.invokeFunction('notebook_chat',[q],{})
      .then(function(r){
        hideTyping();
        var a=r.data['application/json'];
        addMsg(typeof a==='string'?a:JSON.stringify(a),false);
      })
      .catch(function(){
        hideTyping();
        addMsg('Sorry, I encountered an error. Please check your internet connection and try again.',false);
      })
      .finally(function(){btn.disabled=false;inp.focus()});
  };

  window.vcAsk=function(q){inp.value=q;vcSendMsg()};
  window.vcClear=function(){
    msgs.innerHTML='<div class="vc-msg bot"><div class="vc-bbl">&#128075; Chat cleared. Ask me anything!</div></div>';
    chips.style.display='flex';
  };

  inp.addEventListener('keypress',function(e){if(e.key==='Enter')vcSendMsg()});
  inp.focus();
})();
</script>'''))